Evan Kurzman
EHK73
VIX Calc on 10/6/2022 for Trading Volaility Smile Fall 2022 CFEM

In [2]:
import pandas as pd
import datetime as dt
import numpy as np

In [3]:
spx_data = pd.read_excel('./VIX_data_2022_1006.xlsx')

In [25]:
spx_data[spx_data['px_bid().value']==0]

,px_bid().value,Put_Call,Strike,expirey


In [5]:
spx_data['expirey'] = spx_data['ID_SECURITY_DES'].astype(str).str.split().str[2]
spx_data=spx_data.drop(columns='ID_SECURITY_DES')

In [6]:
spx_spot = 3791.77
spot_time = dt.datetime(2022,10,6,9,45,42)
near_term = dt.datetime(2022,11,4,3)
next_term = dt.datetime(2022,11,11,3)

In [106]:
n1 =((near_term - spot_time).days * 24 *60 + (near_term - spot_time).seconds // 60 )
n2 =((next_term - spot_time).days * 24 *60 + (next_term - spot_time).seconds // 60 )
n30 = 30 * 24 *60
n365 = 365*24*60
t1 = ((near_term - spot_time).days * 24 *60 + (near_term - spot_time).seconds // 60 )/ 525600
t2 = ((next_term - spot_time).days * 24 *60 + (next_term - spot_time).seconds // 60 ) / 525600

In [8]:
spx_next_data = spx_data[spx_data['expirey']=="11/11/22"]
spx_next_data

,px_bid().value,Put_Call,Strike,expirey
402,0.05,Put,1000,11/11/22
403,0.05,Put,1200,11/11/22
404,0.05,Put,1400,11/11/22
405,0.10,Put,1600,11/11/22
406,0.20,Put,1800,11/11/22
...,...,...,...,...
634,4.00,Call,4300,11/11/22
635,1.95,Call,4400,11/11/22
636,0.45,Call,4600,11/11/22
637,0.15,Call,4800,11/11/22


In [35]:
spx_near_data = spx_data[spx_data['expirey']=="11/04/22"]
spx_near_data

,px_bid().value,Put_Call,Strike,expirey
0,0.05,Put,1200,11/04/22
1,0.05,Put,1400,11/04/22
2,0.05,Put,1600,11/04/22
3,0.10,Put,1800,11/04/22
4,0.20,Put,2000,11/04/22
...,...,...,...,...
397,1.00,Call,4400,11/04/22
398,0.45,Call,4500,11/04/22
399,0.25,Call,4600,11/04/22
400,0.05,Call,4800,11/04/22


In [10]:
spx_min = 1000
for p in spx_next_data['Strike'].unique():
    m = np.abs(spx_next_data[(spx_next_data['Put_Call']=='Call') & (spx_next_data['Strike'] == p)]['px_bid().value'].values - spx_next_data[(spx_next_data['Put_Call']=='Put') & (spx_next_data['Strike'] == p)]['px_bid().value'].values)
    if m < spx_min:
        spx_min = m
        spx_c_p_2 = spx_next_data[(spx_next_data['Put_Call']=='Call') & (spx_next_data['Strike'] == p)]['px_bid().value'].values - spx_next_data[(spx_next_data['Put_Call']=='Put') & (spx_next_data['Strike'] == p)]['px_bid().value'].values
        spx_forward_f2 = p

spx_min = 1000
for p in spx_near_data['Strike'].unique():
    m = np.abs(spx_near_data[(spx_near_data['Put_Call']=='Call') & (spx_near_data['Strike'] == p)]['px_bid().value'].values - spx_near_data[(spx_near_data['Put_Call']=='Put') & (spx_near_data['Strike'] == p)]['px_bid().value'].values)
    if m < spx_min:
        spx_min = m
        spx_c_p_1 = spx_near_data[(spx_near_data['Put_Call']=='Call') & (spx_near_data['Strike'] == p)]['px_bid().value'].values - spx_near_data[(spx_near_data['Put_Call']=='Put') & (spx_near_data['Strike'] == p)]['px_bid().value'].values
        spx_forward_f1 = p

C:\Users\evank\AppData\Local\Temp\ipykernel_27232\248535646.py:4: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if m < spx_min:
C:\Users\evank\AppData\Local\Temp\ipykernel_27232\248535646.py:12: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if m < spx_min:


In [21]:
print(spx_forward_f1)
print(spx_forward_f2)
print(spx_c_p_1)
print(spx_c_p_2)

3795
3800
[2.40000153]
[-1.59999847]


array([-1.59999847])

In [11]:
f1 = float(spx_forward_f1 + np.exp(.033 * t1) * spx_c_p_1)
f1

3797.406241051229

In [12]:
f2 = float(spx_forward_f2 + np.exp(.033 * t2) * spx_c_p_2)
f2

3798.3948262932604

In [13]:
k01, k02 = max(spx_near_data[spx_near_data['Strike']<f1]['Strike']) , max(spx_next_data[spx_next_data['Strike']<f2]['Strike'])

In [33]:
k01

3795

In [34]:
k02

3790

In [67]:
near_calls = spx_near_data[(spx_near_data['Put_Call']=='Call') & (spx_near_data['Strike'] >= k01)].sort_values('Strike')
near_puts = spx_near_data[(spx_near_data['Put_Call']=='Put') & (spx_near_data['Strike'] <= k01)].sort_values('Strike')
next_calls = spx_next_data[(spx_next_data['Put_Call']=='Call') & (spx_next_data['Strike'] >=k01)].sort_values('Strike')
next_puts = spx_next_data[(spx_next_data['Put_Call']=='Put') & (spx_next_data['Strike'] <= k01)].sort_values('Strike')
k_delta = ((near_calls['Strike'].shift(-1) - near_calls['Strike'].shift()).dropna()/2)
sig_squared_1_t_1_calls = sum((k_delta/near_calls.loc[k_delta.index]['Strike']**2)*np.exp(.033*t1)*near_calls.loc[k_delta.index]['px_bid().value'])
k_delta = ((near_puts['Strike'].shift(-1) - near_puts['Strike'].shift(1)).dropna()/2)
sig_squared_1_t_1_puts = sum((k_delta/near_puts.loc[k_delta.index]['Strike']**2)*np.exp(.033*t1)*near_puts.loc[k_delta.index]['px_bid().value'])
sig_squared_1_t_1_atm = ((near_calls['Strike'].iloc[1] -near_puts['Strike'].iloc[-2])/2)/k01**2*np.exp(.033*t1) *(near_calls[near_calls['Strike']==k01]['px_bid().value'].values + near_puts[near_puts['Strike']==k01]['px_bid().value'].values)/2
sig_squared_1_t_1 = 2/t1*(sig_squared_1_t_1_puts+sig_squared_1_t_1_calls + sig_squared_1_t_1_atm)

In [90]:
sig_squared_1_t_1

array([0.08367652])

In [93]:
next_calls = spx_next_data[(spx_next_data['Put_Call']=='Call') & (spx_next_data['Strike'] >= k02)].sort_values('Strike')
next_puts = spx_next_data[(spx_next_data['Put_Call']=='Put') & (spx_next_data['Strike'] <= k02)].sort_values('Strike')
next_calls = spx_next_data[(spx_next_data['Put_Call']=='Call') & (spx_next_data['Strike'] >=k02)].sort_values('Strike')
next_puts = spx_next_data[(spx_next_data['Put_Call']=='Put') & (spx_next_data['Strike'] <= k02)].sort_values('Strike')
k_delta = ((next_calls['Strike'].shift(-1) - next_calls['Strike'].shift()).dropna()/2)
sig_squared_2_t_1_calls = sum((k_delta/next_calls.loc[k_delta.index]['Strike']**2)*np.exp(.033*t1)*next_calls.loc[k_delta.index]['px_bid().value'])
k_delta = ((next_puts['Strike'].shift(-1) - next_puts['Strike'].shift(1)).dropna()/2)
sig_squared_2_t_1_puts = sum((k_delta/next_puts.loc[k_delta.index]['Strike']**2)*np.exp(.033*t1)*next_puts.loc[k_delta.index]['px_bid().value'])
sig_squared_2_t_1_atm = ((next_calls['Strike'].iloc[1] -next_puts['Strike'].iloc[-2])/2)/k02**2*np.exp(.033*t1) *(next_calls[next_calls['Strike']==k02]['px_bid().value'].values + next_puts[next_puts['Strike']==k02]['px_bid().value'].values)/2
sig_squared_2_t_1 = 2/t1*(sig_squared_2_t_1_puts+sig_squared_2_t_1_calls + sig_squared_2_t_1_atm)

In [94]:
sig_squared_2_t_1

array([0.10689018])

In [97]:
f1

3797.406241051229

In [98]:
sig_squared_1_t_2 = 1/t1 * (f1/k01-1)**2
sig_squared_2_t_2 = 1/t2 * (f2/k02-1)**2
print(sig_squared_1_t_2)
print(sig_squared_2_t_2)

5.109666273913182e-06
5.013604416632351e-05


In [100]:
sig_squared_1 = sig_squared_1_t_1 - sig_squared_1_t_2
sig_squared_1

array([0.08367141])

In [102]:
sig_squared_2 = sig_squared_2_t_1 - sig_squared_2_t_2
sig_squared_2

array([0.10684004])

In [113]:
vix = 100 * ((t1*sig_squared_1*((n2 - n30)/(n2-n1)) + t2*sig_squared_2*((n30 - n1)/(n2-n1)))*n365/n30)**(1/2)

In [117]:
print("VIX: "+ str(round(vix[0],2)))

VIX: 29.79
